# Setup

## Instructions

This notebook utilizes the Kaggle API. Follow these steps in order to get the necessary credentials to continue:

1. Sign up for a Kaggle account at https://www.kaggle.com.
2. Go to the 'Account' tab of your user profile - ```https://www.kaggle.com/{username}/account```.
3. Select 'Create New API Token' under 'API' section.
    - This will trigger the download of kaggle.json, a file containing your API credentials. 
4. Place this file in the location:
    - ~/.kaggle/kaggle.json (for macOS/unix)
    - C:/Users/username/.kaggle/kaggle.json (for Windows) 
    - You can check the exact location, sans drive, with echo %HOMEPATH%). 
    - You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to:
        - $KAGGLE_CONFIG_DIR/kaggle.json (for macOS/unix)
        - %KAGGLE_CONFIG_DIR%\kaggle.json (for Windows)

Note: this notebook uses functions written in Python to query the Kaggle API. **This code will only work for python 3.7 or later**.

## Additional Information

Documentation Guide:
- Kaggle API ([Kaggle](https://www.kaggle.com/docs/api))
- Kaggle API ([GitHub](https://github.com/Kaggle/kaggle-api)) 

## Overview of workflow

#### Import necessary libraries

#### Query #1: query API query based on search terms and search types

Define functions to query API based on search terms and search types:

1.	Function `get_individual_search_output` queries the Kaggle API with the specified search term (e.g., “machine learning”) and search type (must be either “datasets” or “kernels”)
    - Searches across all returned pages
    - Calls function `_convert_string_csv_output_to_dataframe` which converts results from API (strings in semi-structured table) to dataframe format
    - Result is a dataframe (one dataframe per search term/search type combination)


3.	Function `get_all_search_outputs` queries the Kaggle API for all combinations of search terms and search types specified and returns the results as a dictionary of dataframes (one dataframe for each query combination)
    - Calls function `get_individual_search_output`

Run functions for specified search terms and search types. Output is ordered dictionary of dataframes (result #1 "metadata_dict").


#### Query #2: query API for full metadata for hits from initial query

Following the initial query to return high level metadata for hits (resuling in object "ordered_dict"), define functions to retrieve full metadata associated with each object.

4.	Function `_retrieve_object_json` uses the path for each object (each dataframe in result #1 "ordered_dict") to query API for metadata associated with each object. Loads JSON results as dictionary.
    
5.	Function `get_metadata` extracts metadata associated with each object and formats as dataframe
    - Calls function `retrieve_object_json`
    - For each object, appends JSON results from `_retrieve_object_json` to a dataframe (converting to dataframe format)
    - Output is single dataframe for each search query (matching each dataframe in result #1 "ordered_dict")
    
6. Use a `for` loop to put dataframes into an ordered dictionary, matching result #1 "ordered_dict" object
    - Calls function `get_metadata`

Run functions to pull metadata for each object returned by first query. Output is ordered dictionary of dataframes (result #2 "metadata_dict").

We now have a dictionary of results from the first API query (which includes high level metadata such as ID and title) and a dictionary of results with additional metadata for each object, based on results from the first query.

#### Merge results
7.	Function `merge_search_and_metadata_dicts` merges these two dictionaries ("ordered_dict" and "metadata_dict") to a single ordered dictionary and (optional) saves the results as a single csv file
    - Specify `left_on` and `right_on` column names to merge

Run merge function to access full ordered dictionary and optionally save results to csv file.

## Import libraries

In [1]:
# Import kaggle, installing if necessary
try:
    import kaggle
except ImportError as e:
    !pip3 install kaggle
    import kaggle
    
import subprocess # Used to run unix commands
import pandas as pd # For storing/manipulating command data
from io import StringIO # Lets us read csv string output from command into DataFrame
import json # Reading back the metadata files
from tqdm import tqdm # Gives status bar on loop completion
import itertools # For efficient looping over queries
import os # Exporting saved results
from collections import OrderedDict
from utils import flatten_nested_df
from flatten_json import flatten

## Query #1: query API based on search terms and search types

In [2]:
def get_all_search_outputs(search_terms, search_types, flatten_output=False):
    """
    Call the Kaggle API for each search term and search type. 
    Results are retured in results['{term}_{type}'] = df
    
    Params:
    - search_terms (list-like): collection of search terms to query over
    - search_types (list-like): collection of search types to query over
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - results (dict): dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    num_searches = len(search_terms) * len(search_types)
    results = OrderedDict()
    
    for search_term, search_type in itertools.product(search_terms, search_types):
        results[(search_term, search_type)] = get_individual_search_output(search_term, search_type, flatten_output)
        
    return results

In [3]:
def get_individual_search_output(search_term, search_type, flatten_output=False):
    """
    Calls the Kaggle API with the specified search term and returns the search output results.
    
    Params:
    - search_term (str): keyword to seach for
    - search_type (str): objects to search over (must be either datasets or kernels)
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - df (pandas.DataFrame): DataFrame containing the output of the search query
    """
    
    # Make sure our input is valid
    assert isinstance(search_type, str), 'Search term must be a string'
    assert search_type in ('datasets', 'kernels'), 'Search can only be conducted over datasets or kernels'
    
    # Set search variables
    page_idx = 1
    search_output = ''
    cumulative_output = ''
    completion_phrase = f'No {search_type} found\n'
    
    # Pulls the records for a single page of datasets for the given search term
    # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
    search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                     '-s', f'"{search_term}"', 
                                     '-p', str(page_idx)], 
                                    capture_output=True).stdout.decode()

    # Once we no longer see new output, we stop
    while search_output != completion_phrase:
        # Accumulate the output
        cumulative_output = cumulative_output + search_output

        # Increments the page count for searching
        page_idx += 1

        # Pulls the records for a single page of datasets for the given search term
        # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
        search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                         '-s', f'"{search_term}"', 
                                         '-p', str(page_idx)], 
                                        capture_output=True).stdout.decode()

        # Remove header row
        if search_output != completion_phrase:
            search_output = '\r\n'.join(search_output.split('\r\n')[1::])
    
    # Convert search output to DataFrame
    search_df = _convert_string_csv_output_to_dataframe(cumulative_output)
    
    # Rename columns to match names present in metadata df
    search_df.rename(columns={'ref': 'id', 
                              'downloadCount': 'totalDownloads', 
                              'voteCount': 'totalVotes'}, 
                     inplace=True)
        
    if flatten_output:
        search_df = flatten_nested_df(search_df)
    
    return search_df

In [4]:
def _convert_string_csv_output_to_dataframe(output):
    """
    Given a string variable in csv format, returns a Pandas DataFrame
    
    Params:
    - output (str): csv-styled string to be converted
    
    Returns:
    - df (pandas.DataFrame): DataFrame consisting of data from 'output' string variable
    """
    
    # Create DataFrame of results
    output = StringIO(output)
    df = pd.read_csv(output)
    
    return df

#### Run initial API query functions

In [5]:
search_terms = ['korea']
search_types = ['datasets', 'kernels']

In [6]:
search_output_dict = get_all_search_outputs(search_terms, search_types)

In [7]:
sample_key = (search_terms[0], search_types[0])
sample_df = search_output_dict[sample_key]

In [8]:
sample_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,84246,1471,1.000000
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9945,311,1.000000
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1054,24,1.000000
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,591,16,0.970588
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5619,109,0.750000


## Query #2: query API for full metadata for hits from initial query

Note: Unable to find a way to store metadata in memory as opposed to saving file, but this workaround appears to be functional.

In [9]:
def _retrieve_object_json(object_path, flatten_output=False):
    """
    Queries Kaggle for metadata json file & returns the json data as a dictionary
    
    Params:
    - object_path (str): path for the dataset
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - metadata_dict (dict): dictionary containing json metadata
    """
    
    # Download the metadata
    subprocess.run(['kaggle', 'datasets', 'metadata', object_path])

    # Access the metadata and load it in as a dictionary
    with open('dataset-metadata.json') as file:
        json_data = json.load(file)
        
    if flatten_output:
        json_data = flatten(json_data)
        
    return json_data

In [10]:
def get_metadata(object_paths, flatten_output=False):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths (str/list-like): string or list of strings containing the paths for the objects
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - metadata_df (pandas.DataFrame): DataFrame containing metadata for the requested datasets
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
        
    # Run first query
    json_data = _retrieve_object_json(object_paths[0], flatten_output)
        
    # Create DataFrame to store metadata in, using columns found in first query, and then add query info
    metadata_df = pd.DataFrame(columns=json_data.keys(), dtype=object)
    metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    # Pulls metadata information for each dataset found above
    for object_path in tqdm(object_paths[1::]):
        # Download & load the metadata
        json_data = _retrieve_object_json(object_path, flatten_output)

        # Store the metadata into our DataFrame created above
        metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    return metadata_df

#### Run metadata extraction functions

In [11]:
## Extract IDs from DataFrame, and returns as list of strings
metadata_dict = OrderedDict()

for query, df in search_output_dict.items():
    print(f'Retrieving {query} metadata')
    # Create object paths
    _, search_type = query
    object_paths = df.id.values
    
    metadata_dict[query] = get_metadata(object_paths, flatten_output=True)

Retrieving ('korea', 'datasets') metadata


100%|██████████| 178/178 [03:32<00:00,  1.19s/it]


Retrieving ('korea', 'kernels') metadata


100%|██████████| 1019/1019 [19:56<00:00,  1.17s/it] 


In [12]:
metadata_dict[('korea', 'datasets')]

,id,id_no,datasetId,datasetSlug,ownerUser,usabilityRating,totalViews,totalVotes,totalDownloads,title,...,collaborators_15_username,collaborators_15_role,collaborators_16_username,collaborators_16_role,data,collaborators,keywords_5,keywords,keywords_6,keywords_7
0,kimjihoo/coronavirusdataset,527325,527325,coronavirusdataset,kimjihoo,1.000000,492173,1471,84246,[NeurIPS 2020] Data Science for COVID-19 (DS4C),...,minty99,writer,byeongukyu,writer,[],NaN,NaN,NaN,NaN,NaN
1,bappekim/air-pollution-in-seoul,576393,576393,air-pollution-in-seoul,bappekim,1.000000,81584,311,9945,Air Pollution in Seoul,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
2,bappekim/south-korea-visitors,692628,692628,south-korea-visitors,bappekim,1.000000,5923,24,1054,South Korea Visitors,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
3,hongsean/korea-income-and-welfare,1046735,1046735,korea-income-and-welfare,hongsean,0.970588,3085,16,591,Korea Income and Welfare,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
4,bryanpark/korean-single-speaker-speech-dataset,19829,19829,korean-single-speaker-speech-dataset,bryanpark,0.750000,83809,109,5619,Korean Single Speaker Speech Dataset,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,glhr00/rgbt-driveability-segmentation-kaist,1340078,1340078,rgbt-driveability-segmentation-kaist,glhr00,0.562500,47,0,1,RGB-T driveability segmentation,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
175,llkdev/gender-data,231235,231235,gender-data,llkdev,0.125000,342,0,8,Gender Data,...,NaN,NaN,NaN,NaN,[],[],NaN,[],NaN,NaN
176,chenykfrank/immc21,1135726,1135726,immc21,chenykfrank,0.117647,48,0,2,immc21,...,NaN,NaN,NaN,NaN,[],[],NaN,[],NaN,NaN
177,gremmn/gas-prices,1108372,1108372,gas-prices,gremmn,0.117647,174,0,17,gas prices,...,NaN,NaN,NaN,NaN,[],[],NaN,[],NaN,NaN


## Combine results of initial and metadata queries

In [13]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on=None, left_on=None, right_on=None, save=False):
    """
    Merges together all of the search and metadata DataFrames by the given 'on' key
    
    Params:
    - search_dict (dict): dictionary of search output results
    - metadata_dict (dict): dictionary of metadata results
    - on (str/list-like): column name(s) to merge the two dicts on
    - left_on (str/list-like): column name(s) to merge the left dict on
    - right_on (str/list-like): column name(s) to merge the right dict on
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain enough strings (one per query; len(search_terms) * len(search_types))
            
    Returns:
    - df_dict (OrderedDict): OrderedDict containing all of the merged search/metadata dicts
    """

    # Make sure the dictionaries contain the same searches
    assert search_dict.keys() == metadata_dict.keys(), 'Dictionaries must contain the same searches'
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')

    # Merge the DataFrames
    df_dict = OrderedDict()
    for (query_key, search_df), (query_key, metadata_df), save_loc in zip(search_dict.items(), 
                                                                          metadata_dict.items(), 
                                                                          save):

        # Merge small version of "full" dataframe with "detailed" dataframe
        df_all = pd.merge(search_df, metadata_df, on=on, left_on=left_on, right_on=right_on, how='outer')
            
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'kaggle')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure kaggle directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError('Save type must be bool or str')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

#### need to clarify left_on, right_on and if can be incorporated into function

In [14]:
#run merge function
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict)

#### Example output

In [15]:
#results of query #1
output_df = search_output_dict[('korea', 'datasets')]

#results of query #2
metadata_df = metadata_dict[('korea', 'datasets')]

#result of merging datasets into "full" dataframe
full_df = df_dict[('korea', 'datasets')]

In [16]:
output_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,84246,1471,1.000000
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9945,311,1.000000
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1054,24,1.000000
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,591,16,0.970588
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5619,109,0.750000


In [17]:
metadata_df.head()

,id,id_no,datasetId,datasetSlug,ownerUser,usabilityRating,totalViews,totalVotes,totalDownloads,title,...,collaborators_15_username,collaborators_15_role,collaborators_16_username,collaborators_16_role,data,collaborators,keywords_5,keywords,keywords_6,keywords_7
0,kimjihoo/coronavirusdataset,527325,527325,coronavirusdataset,kimjihoo,1.000000,492173,1471,84246,[NeurIPS 2020] Data Science for COVID-19 (DS4C),...,minty99,writer,byeongukyu,writer,[],NaN,NaN,NaN,NaN,NaN
1,bappekim/air-pollution-in-seoul,576393,576393,air-pollution-in-seoul,bappekim,1.000000,81584,311,9945,Air Pollution in Seoul,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
2,bappekim/south-korea-visitors,692628,692628,south-korea-visitors,bappekim,1.000000,5923,24,1054,South Korea Visitors,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
3,hongsean/korea-income-and-welfare,1046735,1046735,korea-income-and-welfare,hongsean,0.970588,3085,16,591,Korea Income and Welfare,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
4,bryanpark/korean-single-speaker-speech-dataset,19829,19829,korean-single-speaker-speech-dataset,bryanpark,0.750000,83809,109,5619,Korean Single Speaker Speech Dataset,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN


In [18]:
full_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating,id_no,datasetId,datasetSlug,...,collaborators_15_username,collaborators_15_role,collaborators_16_username,collaborators_16_role,data,collaborators,keywords_5,keywords,keywords_6,keywords_7
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,84246.0,1471.0,1.000000,527325,527325,coronavirusdataset,...,minty99,writer,byeongukyu,writer,[],NaN,NaN,NaN,NaN,NaN
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9945.0,311.0,1.000000,576393,576393,air-pollution-in-seoul,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1054.0,24.0,1.000000,692628,692628,south-korea-visitors,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,591.0,16.0,0.970588,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5619.0,109.0,0.750000,19829,19829,korean-single-speaker-speech-dataset,...,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
